In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib as mpl
mpl.use('TkAgg')
import matplotlib.pyplot as plt

Погрузка из google drive при работе в Google Collab

In [9]:
# from google.colab import drive
# drive.mount('/content/drive')
# data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/test_dataset.csv')

Подгружаем unprepared.csv из папки проекта при запуске из main.py

In [25]:
from pathlib import Path
import sys
find_root = Path(sys.argv[0]).resolve(strict=True).parents[3]
sys.path.append(str(find_root ))
from service.paths import get_path_to_raw_data
raw_file = get_path_to_raw_data() # текущий RAW_DATA_NAME
df = pd.read_csv(raw_file)
df["success"] = "success"
df.to_csv(raw_file.with_name(raw_file.stem+"_prepped.csv"),float_format='%.5f',mode='w')

объединяем столбцы даты и часа, чтобы можно было их выразить одним числом. Сначала приводим к типу datetime64 из дата первой колонки+временная дельта(часы) второй колонки  
(*результат в отдельной переменной, изначальный датасет не меняется*)

объединяем столбцы даты и часа, чтобы можно было их выразить одним числом. Сначала приводим к типу datetime64 из дата первой колонки+временная дельта(часы) второй колонки  
(*результат в отдельной переменной, изначальный датасет не меняется*)

In [ ]:
def make_dt_col(row_date,row_time):
    return pd.to_datetime(row_date)+pd.to_timedelta(row_time,unit='h')



t_series=df.apply(lambda x: make_dt_col(x['date'],x['time']),axis=1)
t_series

Переводим datetime в секунды и тип float

In [ ]:
#преобразуем datetime во float , измерение в секундах
t_series=t_series.map(pd.Timestamp.timestamp)

задаем периоды на день и на год в секундах , переводим в радианы, и передаем как аргумент в синусоидные функции

In [ ]:
day_in_secs = 24*60*60 
year_in_secs = (365.2425)*day_in_secs


df['Hourly sin'] = np.sin(t_series * (2 * np.pi / day_in_secs))
df['Hourly cos'] = np.cos(t_series * (2 * np.pi / day_in_secs))
df['Daily sin'] = np.sin(t_series * (2 * np.pi / year_in_secs))
df['Daily cos'] = np.cos(t_series * (2 * np.pi / year_in_secs))

Проверяем полученные фичи периодичности по времени(не нужны при использовании временных рядов, так как там даты используются как позиционный аргумент=индекс для сортировки и упорядочения)

Проверяем полученные фичи периодичности по времени(не нужны при использовании временных рядов, так как там даты используются как позиционный аргумент=индекс для сортировки и упорядочения)

In [ ]:
plt.plot(np.array(df['Hourly sin'])[:25])
plt.plot(np.array(df['Hourly cos'])[:25])
plt.xlabel('Time [h]')
plt.title('Time of day signal')

In [ ]:
plt.plot(np.array(df['Daily sin'][:10000]))
plt.plot(np.array(df['Daily cos'][:10000]))
plt.xlabel('Time [d]')
plt.title('Year signal')

Изучение семантики в колонках с погодой. Какие слова и как часто встречаются

In [ ]:
def count_sep_words(df_col,split_by = ','):
    u_w = df_col.str.split(pat=split_by,expand=True) # получаем датафрейм с разбитыми в отдельные колонки сплитами
    u_w = u_w.values.ravel('K') # переводим в одномерный список (numpy.1d-array)
    u_w = u_w[u_w != np.array(None)] # фильтруем пустые сплиты
    u_w = np.char.strip(u_w.astype('str')) # убираем лишние пробелы
    u_w = pd.value_counts(u_w)
    with pd.option_context('display.max_rows', None):
        print(u_w)
    return u_w

count_sep_words(df["weather_fact"])


In [ ]:
print("\nКолонка с предсказанной погодой:")
count_sep_words(df["weather_pred"].dropna().replace("?",""))

In [ ]:
def parse_weather_column(keywords:list,src_column_name:str,target_column_name:str,df):
  '''
  Функция для парсинга отдельных столбцов, ищет вхождение переданных ключевых слов в строке и пишет в новую колонку найденное  слово или пустую строку
  :keywords: -список ключевых слов. Ищутся по порядку перебором , поэтому лучше указывать сначала длинные слова, а потом их подстроки
  (например, если первым ключом будет "ветер", то вместо "ветерок" будет найден "ветер")
  :src_column_name: - название столбца, в котором идет поиск
  :target_column_name: - желаемое название нового столбца
  :df: - объект датафрейма, в котором  исходная колонка
  return копия датафрейма с добавленным новым столбцом(функция не inplace, перезаписывайте при надобности свой датафрейм)
  '''
  def find_kword(row_string):
      nonlocal keywords
      if isinstance(row_string,str):
        for k in keywords:
          if k in row_string:
            return k
      return ""
  found_list = [find_kword(row) for row in df[src_column_name] ]
  new_df = df.copy()
  new_df[target_column_name] = found_list
  return new_df

df  = parse_weather_column(["ветерок","ветрище","ветер"],"weather_fact","wind",df)

df.sample(10)

In [ ]:
df[["weather_fact","wind"]].sample(10)

Переводим колонку ветра в числовой формат с нужной градацией силы ветра

In [ ]:
wind_map = {
    "": 0,
    "ветерок": 0.5,
    "ветер": 1,
    "ветрище":1.5
}
df["wind"] = df["wind"].map(wind_map)
df[["weather_fact","wind"]].sample(10)

In [ ]:
df.to_csv("./data/numeric_features.csv",float_format='%.5f',mode='w')